In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 7, Finished, Available, Finished)

In [6]:
silver_table_path = "abfss://brief_dp700@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Silver.Lakehouse/Tables/dbo/wind_power"

df = spark.read.format("delta").load(silver_table_path)

print(f"📊 Données chargées : {df.count()} lignes")

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 8, Finished, Available, Finished)

📊 Données chargées : 432 lignes


In [7]:
# Créer dim_date avec valeurs uniques
date_dim = (df
    .select("date", "day", "month", "quarter", "year")
    .distinct()
    .withColumnRenamed("date", "date_id")
)

print(f"📅 dim_date : {date_dim.count()} dates uniques")
date_dim.show(5)

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 9, Finished, Available, Finished)

📅 dim_date : 1 dates uniques
+----------+---+-----+-------+----+
|   date_id|day|month|quarter|year|
+----------+---+-----+-------+----+
|2024-06-15| 15|    6|      2|2024|
+----------+---+-----+-------+----+



In [8]:
# Créer dim_time avec valeurs uniques
time_dim = (df
    .select("time", "hour_of_day", "minute_of_hour", "second_of_minute", "time_period")
    .distinct()
    .withColumnRenamed("time", "time_id")
)

print(f"⏰ dim_time : {time_dim.count()} temps uniques")
time_dim.show(5)

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 10, Finished, Available, Finished)

⏰ dim_time : 144 temps uniques
+--------+-----------+--------------+----------------+-----------+
| time_id|hour_of_day|minute_of_hour|second_of_minute|time_period|
+--------+-----------+--------------+----------------+-----------+
|10:10:00|         10|            10|               0|    Morning|
|11:10:00|         11|            10|               0|    Morning|
|18:10:00|         18|            10|               0|    Evening|
|18:30:00|         18|            30|               0|    Evening|
|16:10:00|         16|            10|               0|  Afternoon|
+--------+-----------+--------------+----------------+-----------+
only showing top 5 rows



In [9]:
# Créer dim_turbine avec ID auto-incrémenté
turbine_dim = (df
    .select("turbine_name", "capacity", "location_name", "latitude", "longitude", "region")
    .distinct()
    .withColumn("turbine_id", 
        row_number().over(
            Window.orderBy("turbine_name", "capacity", "location_name")
        )
    )
)

print(f"🌬️ dim_turbine : {turbine_dim.count()} turbines")
turbine_dim.show()

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 11, Finished, Available, Finished)

🌬️ dim_turbine : 3 turbines
+------------+--------+-------------+--------+---------+--------+----------+
|turbine_name|capacity|location_name|latitude|longitude|  region|turbine_id|
+------------+--------+-------------+--------+---------+--------+----------+
|   Turbine A|    2200|   Location 1| 34.0522|-118.2437|Region A|         1|
|   Turbine B|    2000|   Location 2| 36.7783|-119.4179|Region B|         2|
|   Turbine C|    2500|   Location 3| 40.7128|  -74.006|Region C|         3|
+------------+--------+-------------+--------+---------+--------+----------+



In [10]:
# Créer dim_operational_status avec ID
operational_status_dim = (df
    .select("status", "responsible_department")
    .distinct()
    .withColumn("status_id", 
        row_number().over(
            Window.orderBy("status", "responsible_department")
        )
    )
)

print(f"📊 dim_operational_status : {operational_status_dim.count()} statuts")
operational_status_dim.show()

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 12, Finished, Available, Finished)

📊 dim_operational_status : 6 statuts
+--------------------+----------------------+---------+
|              status|responsible_department|status_id|
+--------------------+----------------------+---------+
|               Fault|           Engineering|        1|
|          Inspection|           Maintenance|        2|
|             Offline|            Operations|        3|
|              Online|            Operations|        4|
|Preventive Mainte...|           Maintenance|        5|
|Reactive Maintenance|           Maintenance|        6|
+--------------------+----------------------+---------+



In [11]:
# Joindre pour récupérer les clés étrangères
df_with_keys = (df
    .join(turbine_dim, 
          ["turbine_name", "capacity", "location_name", "latitude", "longitude", "region"], 
          "left")
    .join(operational_status_dim, 
          ["status", "responsible_department"], 
          "left")
)

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 13, Finished, Available, Finished)

In [12]:
# Table de faits avec uniquement clés et mesures
fact_table = (df_with_keys
    .select(
        "production_id",
        col("date").alias("date_id"),
        col("time").alias("time_id"),
        "turbine_id",
        "status_id",
        "wind_speed",
        "wind_direction",
        "energy_produced"
    )
)

print(f"📈 fact_wind_power : {fact_table.count()} mesures")
fact_table.show(10)

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 14, Finished, Available, Finished)

📈 fact_wind_power : 432 mesures
+-------------+----------+--------+----------+---------+----------+--------------+---------------+
|production_id|   date_id| time_id|turbine_id|status_id|wind_speed|wind_direction|energy_produced|
+-------------+----------+--------+----------+---------+----------+--------------+---------------+
|         6049|2024-06-15|00:00:00|         1|        4|      5.74|            SW|        1783.39|
|         6050|2024-06-15|00:00:00|         2|        2|     23.92|             E|            0.0|
|         6051|2024-06-15|00:00:00|         3|        4|      5.72|            NW|        1651.84|
|         6052|2024-06-15|00:10:00|         1|        4|     13.02|             E|        1351.88|
|         6053|2024-06-15|00:10:00|         2|        4|      8.77|             E|         931.99|
|         6054|2024-06-15|00:10:00|         3|        4|     23.31|             W|        1459.19|
|         6055|2024-06-15|00:20:00|         1|        4|      2.35|          

In [13]:
base_path = "abfss://brief_dp700@onelake.dfs.fabric.microsoft.com/LH_Wind_Power_Gold.Lakehouse/Tables/dbo"

gold_date_dim_path = f"{base_path}/dim_date"
gold_time_dim_path = f"{base_path}/dim_time"
gold_turbine_dim_path = f"{base_path}/dim_turbine"
gold_operational_status_dim_path = f"{base_path}/dim_operational_status"
gold_fact_table_path = f"{base_path}/fact_wind_power"

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 15, Finished, Available, Finished)

In [14]:
# Sauvegarder toutes les tables
date_dim.write.format("delta").mode("overwrite").save(gold_date_dim_path)
time_dim.write.format("delta").mode("overwrite").save(gold_time_dim_path)
turbine_dim.write.format("delta").mode("overwrite").save(gold_turbine_dim_path)
operational_status_dim.write.format("delta").mode("overwrite").save(gold_operational_status_dim_path)
fact_table.write.format("delta").mode("overwrite").save(gold_fact_table_path)

print("✅ Toutes les tables créées dans Gold !")

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 16, Finished, Available, Finished)

✅ Toutes les tables créées dans Gold !


In [17]:
from pyspark.sql.functions import count,when

print("=== 🔍 Vérification intégrité référentielle ===\n")

# Vérifier les dates
fact_dates = fact_table.select("date_id").distinct().count()
dim_dates = date_dim.count()
print(f"📅 Dates - Fait: {fact_dates}, Dim: {dim_dates}")

# Vérifier les turbines
fact_turbines = fact_table.select("turbine_id").distinct().count()
dim_turbines = turbine_dim.count()
print(f"🌬️ Turbines - Fait: {fact_turbines}, Dim: {dim_turbines}")

# Vérifier les clés nulles
null_check = fact_table.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["date_id", "time_id", "turbine_id", "status_id"]
])
print("\n❌ Clés nulles dans fact :")
null_check.show()

StatementMeta(, 3ec0ff8f-6ed3-4122-a73b-c13dd3b26b85, 19, Finished, Available, Finished)

=== 🔍 Vérification intégrité référentielle ===

📅 Dates - Fait: 1, Dim: 1
🌬️ Turbines - Fait: 3, Dim: 3

❌ Clés nulles dans fact :
+-------+-------+----------+---------+
|date_id|time_id|turbine_id|status_id|
+-------+-------+----------+---------+
|      0|      0|         0|        0|
+-------+-------+----------+---------+

